In [ ]:
#!pip install gym-tetris nes_py 
!pip install git+https://github.com/DLR-RM/stable-baselines3
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib/
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]
!pip install moviepy
#!pip install envpool

  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-pemz1lkc
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-pemz1lkc
  Resolved https://github.com/DLR-RM/stable-baselines3 to commit 4fcda6b2d453a179c615162a36624d654e3456b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 16.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for stable-baselines3: filename=stable_baselines3-2.0.0a11-py3-none-any.whl size=177970 sha256=56c758301482f473b7507fba3c2654de1c7a30b98f976c6bb5eee1f5e3067c41
  Stored in directory: /tmp/pip-ephem-wheel-cache-02foiw1e/wheels/3b/24/65/bc2794face336930a72bdbe36faf5aad6e2352b3d1dec310ca
Successfully built stable-baselines3
  Attempting uninstall: gymnasium
    Found existing installation: Gymnasium 0.26.3
    Uninstalling Gymnasium-0.26.3:

In [ ]:
#import gym_tetris
import gymnasium 
#import envpool

#from nes_py.wrappers import JoypadSpace
#from gym_tetris.actions import SIMPLE_MOVEMENT, MOVEMENT  # exists MOVEMENT, with more moves
import torch
import numpy as np
from gymnasium.wrappers import FrameStack, GrayScaleObservation, FrameStack, ResizeObservation, AtariPreprocessing, RecordEpisodeStatistics
from gymnasium.vector import VectorEnv
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecVideoRecorder, VecEnv, VecMonitor, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.atari_wrappers import AtariWrapper, MaxAndSkipEnv, WarpFrame, FireResetEnv, ClipRewardEnv, NoopResetEnv, EpisodicLifeEnv
from stable_baselines3 import PPO, DQN, TD3, SAC
from stable_baselines3.ppo.policies import CnnPolicy
from sb3_contrib import QRDQN
from matplotlib import pyplot as plt


In [ ]:
# Atari wrapper form baselines3

env = gymnasium.make("ALE/Breakout-v5", render_mode="rgb_array")

env = NoopResetEnv(env, noop_max=30)
env = MaxAndSkipEnv(env, skip=4)
env = FireResetEnv(env)
#env = EpisodicLifeEnv(env)
env = ClipRewardEnv(env)
env = ResizeObservation(env, (84, 84))
env = GrayScaleObservation(env)
env = FrameStack(env, 1)

#env = DummyVecEnv([lambda: env])
#env = VecFrameStack(env, 1, channels_order="first")
#env = VecNormalize(env, training=True, norm_obs=True, clip_obs= 1, norm_reward=False, gamma=0.99, epsilon=1e-08)

In [ ]:
env = gymnasium.make("ALE/Breakout-v5", render_mode="rgb_array")
#env = FireResetEnv(env)
#env = NoopResetEnv(env, noop_max=30)
#env = MaxAndSkipEnv(env, skip=4)
#env = ClipRewardEnv(env)
#env = ResizeObservation(env, (84, 84))
#env = GrayScaleObservation(env)
#env = FrameStack(env, 1)

env = AtariPreprocessing(env, noop_max=30, frame_skip=1, screen_size=84,
                        terminal_on_life_loss=False, grayscale_obs=True,
                        scale_obs=False)
env = FireResetEnv(env)
#env = EpisodicLifeEnv(env)   #añadido
env = MaxAndSkipEnv(env, skip=4)
env = ClipRewardEnv(env)
env = FrameStack(env, 32)
env = RecordEpisodeStatistics(env=env, deque_size=200)
env = DummyVecEnv([lambda: env])

In [ ]:
#CnnPolicy(action_space=env.action_space, observation_space=env.observation_space, lr_schedule=lambda f: 2.4e-5*f, 
#          ortho_init=True, normalize_images=False)

#model = PPO("CnnPolicy", env, verbose=1, learning_rate=lambda f: 2.4e-4*f, n_steps=128, batch_size=128,
#                    gamma=0.99, gae_lambda=0.95, clip_range=0.1)


policy_kwargs = dict(n_quantiles=50)
model = QRDQN("CnnPolicy", env, learning_rate=lambda f: 2.4e-4*f, batch_size=128,
                     gamma=0.99, policy_kwargs=policy_kwargs, verbose=1, buffer_size=20000)
model.learn(total_timesteps=3_000_000)

In [ ]:
  """
    Policy class for QR-DQN when using images as input.
    :param observation_space: Observation space
    :param action_space: Action space
    :param lr_schedule: Learning rate schedule (could be constant)
    :param n_quantiles: Number of quantiles
    :param net_arch: The specification of the network architecture.
    :param activation_fn: Activation function
    :param features_extractor_class: Features extractor to use.
    :param normalize_images: Whether to normalize images or not,
         dividing by 255.0 (True by default)
    :param optimizer_class: The optimizer to use,
        ``th.optim.Adam`` by default
    :param optimizer_kwargs: Additional keyword arguments,
        excluding the learning rate, to pass to the optimizer
    """

In [ ]:
############ RecurrentPPO ##############
from sb3_contrib import RecurrentPPO

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy

model = RecurrentPPO("MlpLstmPolicy", env, verbose=1, learning_rate=lambda f: 2.4e-4*f, n_steps=128, batch_size=128,
                    gamma=0.99, gae_lambda=0.95, clip_range=0.1)
model.learn(2_000_000)


In [ ]:
############ EVALUACION DEL REWARD DEL MODELO #################

from stable_baselines3.common.evaluation import evaluate_policy

vec_env = model.get_env()
mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=20, warn=False)
print(mean_reward)

In [ ]:
############ RecurrentPPO ##############
import imageio
import numpy as np

obs = env.reset()
video_length = 5_000
# cell and hidden state of the LSTM
lstm_states = None
num_envs = 1
# Episode start signals are used to reset the lstm states
episode_starts = np.ones((num_envs,), dtype=bool)
images = []

img = model.env.render(mode="rgb_array")
#img = model.env.render()
for i in range(video_length + 1):
    if i ==video_length:
        print("TERMINA")
    images.append(img)
    action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True)
    obs, rewards, done, info = env.step(action)
    episode_starts = done
    #img = model.env.render()
    img = model.env.render(mode="rgb_array")
    #env.render()
    # VecEnv resets automatically
    if done:
        obs = env.reset()
    
imageio.mimsave("1.2M_lstm_breakout_5k_vec1_128stepbatch.gif", 
                [np.array(img) for i, img in enumerate(images) if i%2 == 0], 
                duration=20)
env.close()

In [ ]:
model.save('5M_vec_1stack_128steps_128bsize')

In [ ]:
model = PPO.load("/content/drive/MyDrive/RL/modelos/breakout_250k_1stack_128steps_128bsize.zip")

In [ ]:
# GIF from images

import imageio
import numpy as np

env = model.get_env()

# Record the video starting at the first step
video_folder = "/kaggle/working/videos"
video_length = 5_000
#env = VecVideoRecorder(env, video_folder,
 #                      record_video_trigger=lambda x: x == 0, video_length=video_length,
 #                      name_prefix=f"random-agent-{'Breakout-v5'}")

images = []

obs = env.reset()

img = model.env.render(mode="rgb_array")
#img = model.env.render()
for i in range(video_length + 1):
    if i ==video_length:
        print("TERMINA")
    images.append(img)
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #img = model.env.render()
    img = model.env.render(mode="rgb_array")
    #env.render()
    # VecEnv resets automatically
    if done:
        obs = env.reset()
    
imageio.mimsave("850k_episodic_breakout_5k_vec32_128stepbatch.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], duration=20)
env.close()

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#################### RESULTADOS ###############################

######################################################
-----------------------------------------
| time/                   |             |
|    fps                  | 116         |
|    iterations           | 7846        |
|    time_elapsed         | 8611        |
|    total_timesteps      | 1004288     |
| train/                  |             |
|    approx_kl            | 0.016857605 |
|    clip_fraction        | 0.175       |
|    clip_range           | 0.1         |
|    entropy_loss         | -0.587      |
|    explained_variance   | 0.911       |
|    learning_rate        | 0.000192    |
|    loss                 | 0.0637      |
|    n_updates            | 78450       |
|    policy_gradient_loss | -0.0215     |
|    value_loss           | 0.288       |
-----------------------------------------

env = gymnasium.make("ALE/Breakout-v5", render_mode="rgb_array")

env = AtariPreprocessing(env, noop_max=30, frame_skip=1, screen_size=84,
                        terminal_on_life_loss=False, grayscale_obs=True,
                        scale_obs=False)
env = FireResetEnv(env)
env = MaxAndSkipEnv(env, skip=4)
env = ClipRewardEnv(env)
env = FrameStack(env, 1)
env = DummyVecEnv([lambda: env])
#------------------------------------------------

model = PPO("CnnPolicy", env, verbose=1, learning_rate=lambda f: 2.4e-4*f, n_steps=128, batch_size=128,
                    gamma=0.99, gae_lambda=0.95, clip_range=0.1)
model.learn(total_timesteps=5_000_000

######################################################
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 203          |
|    ep_rew_mean          | 14.4         |
| time/                   |              |
|    fps                  | 136          |
|    iterations           | 8081         |
|    time_elapsed         | 7577         |
|    total_timesteps      | 1034368      |
| train/                  |              |
|    approx_kl            | 0.0056244344 |
|    clip_fraction        | 0.0805       |
|    clip_range           | 0.1          |
|    entropy_loss         | -0.538       |
|    explained_variance   | 0.321        |
|    learning_rate        | 0.00019      |
|    loss                 | -0.00809     |
|    n_updates            | 80800        |
|    policy_gradient_loss | -0.00372     |
|    value_loss           | 0.0025       |
------------------------------------------ 
#env = gymnasium.make("ALE/Breakout-v5", render_mode="rgb_array")

#env = NoopResetEnv(env, noop_max=30)
#env = MaxAndSkipEnv(env, skip=4)
#env = FireResetEnv(env)
#env = ClipRewardEnv(env)
#env = ResizeObservation(env, (84, 84))
#env = GrayScaleObservation(env)
#env = FrameStack(env, 1)
#------------------------------------------------
#model = PPO("CnnPolicy", env, verbose=1, learning_rate=lambda f: 2.4e-4*f, n_steps=128, batch_size=128,
 #                   gamma=0.99, gae_lambda=0.95, clip_range=0.1)
#model.learn(total_timesteps=5_000_000)
######################################################